In [1]:
%load_ext autoreload
%autoreload 2

%load_ext line_profiler

In [2]:
import pandas as pd
import numpy as np

import  matplotlib.pyplot as plt
import pickle

from chemistry import Molecule
from util import score
from pprint import pprint


In [3]:
with open('../data/molecules_enh.pickle', 'rb') as f:
    molecules = pickle.load(f)

In [4]:
structures = pd.read_feather('../data/structures_enh.feather')
molecules_df = pd.read_feather('../data/molecules.feather')
labelled = pd.read_feather('../data/train.feather')
unlabelled = pd.read_feather('../data/test.feather')

/home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/pyarrow/pandas_compat.py:752: FutureWarning:

.labels was deprecated in version 0.24.0. Use .codes instead.



In [5]:
dipole_moments = pd.read_feather('../data/dipole_moments.feather')
magnetic_shielding_tensors = pd.read_feather('../data/magnetic_shielding_tensors.feather')
mulliken_charges = pd.read_feather('../data/mulliken_charges.feather')
potential_energy = pd.read_feather('../data/potential_energy.feather')
scalar_coupling_contributions = pd.read_feather('../data/scalar_coupling_contributions.feather')

In [6]:
labelled_enh = labelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
labelled_enh.head(32)

,id,molecule_name,atom_index_0,atom_index_1,type,scalar_coupling_constant,nC,nF,nH,nN,nO,nCNOF,nCN,nOF,nAtoms
0,0,dsgdb9nsd_000001,1,0,1JHC,84.807602,1,0,4,0,0,1,1,0,5
1,1,dsgdb9nsd_000001,1,2,2JHH,-11.257000,1,0,4,0,0,1,1,0,5
2,2,dsgdb9nsd_000001,1,3,2JHH,-11.254800,1,0,4,0,0,1,1,0,5
3,3,dsgdb9nsd_000001,1,4,2JHH,-11.254300,1,0,4,0,0,1,1,0,5
4,4,dsgdb9nsd_000001,2,0,1JHC,84.807404,1,0,4,0,0,1,1,0,5
5,5,dsgdb9nsd_000001,2,3,2JHH,-11.254100,1,0,4,0,0,1,1,0,5
6,6,dsgdb9nsd_000001,2,4,2JHH,-11.254800,1,0,4,0,0,1,1,0,5
7,7,dsgdb9nsd_000001,3,0,1JHC,84.809303,1,0,4,0,0,1,1,0,5
8,8,dsgdb9nsd_000001,3,4,2JHH,-11.254300,1,0,4,0,0,1,1,0,5
9,9,dsgdb9nsd_000001,4,0,1JHC,84.809502,1,0,4,0,0,1,1,0,5


In [7]:
unlabelled_enh = unlabelled.merge(molecules_df, left_on='molecule_name', right_on='molecule_name')
unlabelled_enh.head(32)

,id,molecule_name,atom_index_0,atom_index_1,type,nC,nF,nH,nN,nO,nCNOF,nCN,nOF,nAtoms
0,4658147,dsgdb9nsd_000004,2,0,2JHC,2,0,2,0,0,2,2,0,4
1,4658148,dsgdb9nsd_000004,2,1,1JHC,2,0,2,0,0,2,2,0,4
2,4658149,dsgdb9nsd_000004,2,3,3JHH,2,0,2,0,0,2,2,0,4
3,4658150,dsgdb9nsd_000004,3,0,1JHC,2,0,2,0,0,2,2,0,4
4,4658151,dsgdb9nsd_000004,3,1,2JHC,2,0,2,0,0,2,2,0,4
5,4658152,dsgdb9nsd_000015,3,0,1JHC,2,0,6,0,1,3,2,1,9
6,4658153,dsgdb9nsd_000015,3,2,3JHC,2,0,6,0,1,3,2,1,9
7,4658154,dsgdb9nsd_000015,3,4,2JHH,2,0,6,0,1,3,2,1,9
8,4658155,dsgdb9nsd_000015,3,5,2JHH,2,0,6,0,1,3,2,1,9
9,4658156,dsgdb9nsd_000015,4,0,1JHC,2,0,6,0,1,3,2,1,9


In [ ]:
len(labelled), len(labelled_enh)

In [ ]:
from models import NNModel, partition_data

#data_df = labelled[labelled.type == '1JHC'].head(10)
data_df = labelled_enh.head(1000)
train_df, test_df = partition_data(data_df, train_frac=1)
train_df = train_df.copy()
        
model = NNModel(model_args = dict(molecules=molecules,
                                  structures=structures))
#model.corr(train_df, train_df)
model.setup_data(train_df, train_df)

In [ ]:
model.structures.columns

In [ ]:
model.input_df.columns

In [ ]:
model.input_df.dtypes

In [ ]:
labelled_enh.head()

In [ ]:
import itertools

In [ ]:
df = labelled_enh.head(32)

In [ ]:
#df[['molecule_name']].merge(dipole_moments, left_on='molecule_name', right_on='molecule_name')


In [8]:
from types import SimpleNamespace
nn_args = SimpleNamespace(dipole_moments=dipole_moments,
                          magnetic_shielding_tensors=magnetic_shielding_tensors,
                          mulliken_charges=mulliken_charges,
                          potential_energy=potential_energy,
                          scalar_coupling_contributions=scalar_coupling_contributions)

In [39]:
### from models import NNModel
from models import partition_data, NNModel

# import tensorflow as tf
# from tensorflow.keras import backend as tfbe

# tf.reset_default_graph()

# atom_count_ranges = [(1, 10), (11, 15), (16, 20), (21, 25), (26, 30)]
# C_count_ranges = [(1, 5), (6, 6), (7, 7), (8, 9)]
coupling_types = sorted(labelled_enh.type.unique())[0:1]
def test(data, count=5000):
    global model
    
    plt.figure(figsize=(25, 25))
    
    for i, t in enumerate(coupling_types, 1):    
        data_df = data[data.type == t].head(count)

        train_df, test_df = partition_data(data_df)

        if len(train_df) < 10 or len(test_df) < 10:
            continue

        print(f'Training {len(train_df)} samples for {t}')

        model = NNModel(dict(molecules=molecules, 
                             structures=structures),
                        nn_args)
        model.fit(train_df, train_df)
        
        print(f'Evaluating {len(test_df)} samples')
        output, score = model.evaluate(test_df, test_df)
        print(f'{t} score: {score} (trained on {len(train_df)} elements)')


        #print(output.shape, test_df.values.shape)
        #pprint(list(zip(output[0:20, 0], test_df.scalar_coupling_constant[0:20])))

        plt.subplot(3, 3, i)
        plt.plot(test_df.scalar_coupling_constant, output, '*')
        mn = min(test_df.scalar_coupling_constant.min(), output.min())
        mx = min(test_df.scalar_coupling_constant.max(), output.max())

        plt.plot([mn, mx], [mn, mx])
        plt.title(t)
                        
    plt.show()
        
test(labelled_enh, 5000)


Training 3500 samples for 1JHC


W0815 23:16:25.988433 139771570575168 deprecation.py:506] From /home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/tensorflow/python/keras/initializers.py:143: calling RandomNormal.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0815 23:16:30.820236 139771570575168 deprecation.py:506] From /home/gautham/apps/anaconda3/envs/kaggle/lib/python3.7/site-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/100
3500/3500 [==============================] - 9s 3ms/sample - loss: 6.1630 - merged_loss: 1.0422 - output_dipole_moments_loss: 1.1297 - output_magnetic_shielding_tensors_loss: 1.0940 - output_mulliken_charges_loss: 0.9724 - output_potential_energy_loss: 0.9899 - output_scalar_coupling_contributions_loss: 0.9200
Epoch 2/100
3500/3500 [==============================] - 1s 237us/sample - loss: 4.8385 - merged_loss: 0.7071 - output_dipole_moments_loss: 0.9280 - output_magnetic_shielding_tensors_loss: 0.9343 - output_mulliken_charges_loss: 0.7770 - output_potential_energy_loss: 0.8021 - output_scalar_coupling_contributions_loss: 0.6858
Epoch 3/100
3500/3500 [==============================] - 1s 236us/sample - loss: 4.4276 - merged_loss: 0.5870 - output_dipole_moments_loss: 0.8577 - output_magnetic_shielding_tensors_loss: 0.8928 - output_mulliken_charges_loss: 0.7317 - output_potential_energy_loss: 0.7715 - output_scalar_coupling_contributions_loss: 0.5842
Epoch 4/100
3500/3500 [=

<Figure size 1800x1800 with 0 Axes>

In [ ]:
mo

In [ ]:
from models import partition_data, NNModel

def train(data):
    models = {}
    for t in sorted(data.type.unique()):
        train_df = data[data.type == t]
        print(f'Training {len(train_df)} samples for {t}')
        model = NNModel(dict(molecules=molecules, 
                             structures=structures),
                        None)
        models[t] = model
        model.fit(train_df, train_df)
    
    return models

models = train(labelled_enh)

In [ ]:
unlabelled_enh.shape

In [ ]:
def predict(data, models):
    out_df = None
    
    for t in sorted(data.type.unique()):
        predict_df = data[data.type == t]
        print(f'Predicting {len(predict_df)} samples for {t}')
        output = models[t].predict(predict_df)
        
        id = predict_df['id']
        out_df_coupling = pd.DataFrame(data={'id':id, 'scalar_coupling_constant':output.flatten()}, index=predict_df.index)
        
        if out_df is None:
            out_df = out_df_coupling
        else:
            out_df = out_df.append(out_df_coupling).sort_index()

    return out_df.sort_index()
    
#%prun -s cumulative f(unlabelled.head(10000))
prediction = predict(unlabelled_enh, models)

In [ ]:
prediction.head()

In [ ]:
prediction.to_csv('../data/pred.csv',index=False)

In [ ]:
import pickle
with open('nn.pickle', 'wb') as f:
    pickle.dump(models, f)

In [ ]:
models